In [49]:
# import important libraries
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from math import floor
import os
import torchvision.models
import copy
use_cuda = True #enable usage of cuda GPU
torch.manual_seed(1000)

In [70]:
# need to upload your initial video, your Best_Model and manually add files frames (not used) and newframes (used)
VIDEO_LOADER_PATH="/content/secondtrashvid.mp4"
VIDEO_MODDED_PATH="/content/secondtrashdemowithlabels.mp4"
MODEL_PATH="/content/Best_Model"
frames_path="/content/frames/"
new_frames_path="/content/newframes/"

In [51]:
classes = ['compost_food',
 'compost_other',
 'garbage',
 'recy_bags',
 'recy_bottles',
 'recy_cans',
 'recy_cups',
 'recy_other_containers',
 'recy_paper',
 'recy_styrofoam']


In [71]:
#get frames of video
import cv2
vidcap = cv2.VideoCapture(VIDEO_LOADER_PATH)
frames=[]
def getFrame(sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    if hasFrames:
        image=cv2.resize(image,(224,224))
        frames.append(image)    # save frame as JPG file
        
        cv2.imwrite(frames_path+"image"+str(count)+".jpg", image)
    return hasFrames
sec = 0
frameRate = .04 #//it will capture image in each 0.5 second
count=1
success = getFrame(sec)
while success:
    count = count + 1
    sec = sec + frameRate
    sec = round(sec, 2)
    success = getFrame(sec)

In [5]:

font = cv2.FONT_HERSHEY_SIMPLEX


In [6]:
resnet152 = torchvision.models.resnet152(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.cache/torch/hub/checkpoints/resnet152-b121ed2d.pth


In [7]:
class Waste_Classifier_One_Linear_Layer(nn.Module):
  def __init__(self, name="WasteNet", hidden_size=256*6*6):
    super(Waste_Classifier_One_Linear_Layer, self).__init__()
    self.name = name
    self.fc1 = nn.Linear(hidden_size, 10)
    self.hidden_size = hidden_size

  def forward(self, x):
    x = x.view(-1, self.hidden_size) #flatten feature data
    x = self.fc1(x)
    return x

In [8]:
resnet152 = torchvision.models.resnet152(pretrained=True)

In [ ]:

resnet152 = nn.Sequential(*(list(resnet152.children())[0:9]))
resnet152.eval()

In [11]:
model_fc=Waste_Classifier_One_Linear_Layer(name="res152_best", hidden_size=2048*1*1)
if use_cuda and torch.cuda.is_available():
  model_fc = model_fc.cuda()

In [12]:
model_fc.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

In [13]:
from PIL import Image
import torchvision.transforms.functional as TF

In [14]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
transformations = transforms.Compose([                                     
  transforms.Resize((224,224)),
  transforms.ToTensor(),
  transforms.Normalize(mean, std)  
])

In [64]:
#get predicted values and add text to frame
from PIL import Image
from matplotlib import cm
def get_predication(frame):
  # print(type(frame))
  image1 = Image.fromarray(frame)
  

  # print(image1)
  # cv2_imshow(image1)
  # image1=torch.from_numpy(image1)
  input_image1 = transformations(image1)
  if use_cuda and torch.cuda.is_available():
    input_image1 = input_image1.cuda()
    model_cuda = resnet152.cuda()
    pred1 = model_cuda(input_image1[None, ...])
  else:
    pred1 = resnet152(input_image1[None, ...])
  pred1_out = model_fc(pred1)

  # print()
  pred=pred1_out.softmax(1).tolist()
  pred=[round(num,2) for num in pred[0]]
  # print(list(zip(classes,pred)))
  pred,cls= pred1_out.softmax(1).max(1, keepdim=True)[0],classes[pred1_out.softmax(1).max(1, keepdim=True)[1]]
  # print(image1,pred,cls)
  frame=cv2.putText(frame, "class "+cls+" predn: "+str(round(pred.tolist()[0][0],2)), (50,50), font, .3, 
            (0, 0, 0),1,
            cv2.LINE_4)
  return frame

In [72]:
#run through frames and call cell above. 
count=0
# print(type(frames[0]))
frame_array=frames.copy()
for frame in frame_array:
  # image1 = cv2.imread(frames_path+frame)
  # cv2_imshow(image1)
  
  # pred , cls = 

  newimg=get_predication(frame)
  # plt.imshow(newimg)
  # cv2_imshow(newimg)
  cv2.imwrite(new_frames_path+"image"+str(count)+".jpg", newimg)
  count+=1


In [73]:
#convert new frames to video which you can then download
import cv2
import numpy as np
import os
from os.path import isfile, join


fps = 24
frame_array=[]
for frame in frames:
    # filename=VI + files[i]
    #reading each files
    frame=cv2.resize(frame,(224,224))
    size = (224,224)
    
    #inserting the frames into an image array
    frame_array.append(frame)
out = cv2.VideoWriter(VIDEO_MODDED_PATH,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
for i in range(len(frame_array)):
    # writing to a image array
    out.write(frame_array[i])
out.release()